In [1]:
#라이브러리 로드
import numpy as np
import pandas as pd

In [2]:
#데이터프레임 로드(1달,3달,5달치 예측(가중치 미 반영))
df_month_1 = pd.read_csv("예매건수실전예측(1달).csv")
df_month_3 = pd.read_csv("예매건수실전예측(3달).csv")
df_month_5 = pd.read_csv("예매건수실전예측(5달).csv")

In [19]:
#최댓값 확인
print(np.max(df_month_1['ticketing_count_forecasted']))
print(np.max(df_month_3['ticketing_count_forecasted']))
print(np.max(df_month_5['ticketing_count_forecasted']))

41074
41074
41074


In [20]:
#최솟값 확인
print(np.min(df_month_1['ticketing_count_forecasted']))
print(np.min(df_month_3['ticketing_count_forecasted']))
print(np.min(df_month_5['ticketing_count_forecasted']))

3659
3659
3659


In [75]:
df_month_3['Unnamed: 0']

0     2021/09/01
1     2021/09/02
2     2021/09/03
3     2021/09/04
4     2021/09/05
         ...    
86    2021/11/26
87    2021/11/27
88    2021/11/28
89    2021/11/29
90    2021/11/30
Name: Unnamed: 0, Length: 91, dtype: object

In [76]:
df_value = df_month_3['ticketing_count_forecasted']

In [78]:
df_value.index = df_month_3['Unnamed: 0']

In [79]:
df_value

Unnamed: 0
2021/09/01    16962
2021/09/02    13949
2021/09/03    18741
2021/09/04    38966
2021/09/05    28830
              ...  
2021/11/26    21903
2021/11/27    38452
2021/11/28    28710
2021/11/29     9057
2021/11/30    10828
Name: ticketing_count_forecasted, Length: 91, dtype: int64

In [80]:
#가중치 반영해주는 함수 선언
def apply_weight(df,n):
    #코로나의 추세가 지나치게 많이 반영된 시점을 경계로 가중치를 다르게 적용하기 위해
    #두 기간으로 분리
    pre = df["ticketing_count_forecasted"][:136]
    post = df["ticketing_count_forecasted"][136:]
    date = df['Unnamed: 0']
    #기간 내 최댓값과 최솟값 확인
    max_value = np.max(df['ticketing_count_forecasted'])
    min_value = np.min(df['ticketing_count_forecasted'])

    #현상유지 시나리오 결과 리스트
    same=[]
    #약해진 거리두기 시나리오 결과 리스트
    weaker=[]
    #강해진 거리두기 시나리오 결과 리스트
    stronger=[]

    #앞 기간에 대한 가중치 적용 후 시나리오별 수치 저장
    #예측 결과에 비해 실제 예측값이 최대 최소간 차이가 큰 점을 반영함
    #2020년 말에 급증했던 코로나 확진자 수로 인해 평년보다 과도하게 급격하게 감소하는 경향성을 완화해서 반영함
    
    for index,i in enumerate(pre):
        if i >= max_value * 0.9:
            same.append(i*1.1)
            stronger.append(i*0.9)
            iw = i + index*10
            weaker.append(iw*1.3)
        elif i <= min_value*1.6:
            same.append(i*0.8)
            stronger.append(i*0.7)
            weaker.append(i*0.85)
        else:
            same.append(i*0.9)
            stronger.append(i*0.75)
            weaker.append(i)

    for index,i in enumerate(post):
        if i >= max_value * 0.8:
            same.append(i*1.2)
            stronger.append(i*0.85)
            iw = i + (index+136)*10
            weaker.append(iw*1.3)
        elif i <= min_value*1.6:
            same.append(i*0.8)
            stronger.append(i*0.7)
            weaker.append(i*0.85)
        else:
            same.append(i*0.9)
            stronger.append(i*0.75)
            weaker.append(i)

    same = pd.Series(np.round(same),index=date)
    stronger = pd.Series(np.round(stronger),index=date)
    weaker = pd.Series(np.round(weaker),index=date)
    
    same.to_csv(f"최종예측{n}달(현재유지).csv")
    stronger.to_csv(f"최종예측{n}달(강해진거리두기).csv")
    weaker.to_csv(f"최종예측{n}달(약해진거리두기).csv")
    return(same,stronger,weaker)

In [81]:
month_1_same, month_1_stronger, month_1_weaker, = apply_weight(df_month_1,1)
month_3_same, month_3_stronger, month_3_weaker, = apply_weight(df_month_3,3)
month_5_same, month_5_stronger, month_5_weaker, = apply_weight(df_month_5,5)

In [63]:
month_1_stronger

array([12722., 10462., 14056., 35069., 21622.,  3403.,  5301., 12800.,
       10472., 13095., 36700., 23089.,  2561.,  6796., 12781., 11170.,
       13682., 36967., 22588.,  3850.,  6652., 12970., 10951., 14302.,
       36577., 22423.,  3959.,  6925., 12446., 10894.])

In [65]:
month_1_weaker

array([16962., 13949., 18741., 50695., 28830.,  4132.,  7068., 17066.,
       13963., 17460., 53141., 30785.,  3110.,  9061., 17041., 14894.,
       18243., 53617., 30118.,  4675.,  8869., 17293., 14601., 19070.,
       53145., 29897.,  4808.,  9233., 16595., 14526.])